# 01 — Data Preparation

**Goal:** Build a clean, merged **country–year panel** for Europe from five Eurostat datasets:
- House Price Index (HPI)
- Net Earnings
- Unemployment rate
- Inflation (HICP index)
- GDP per capita

**This notebook will:**
1) Load tidy CSVs exported by our ingestion script  
2) Standardize country names (simple mapping)  
3) Ensure types & basic ranges  
4) Merge to a single panel (country, year)  
5) Run sanity checks (missingness, duplicates, outliers)  
6) Save `merged_panel.{csv,parquet}` in `data/processed/`


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 20)
plt.rcParams["figure.figsize"] = (7, 5)

PROC = Path("../data/processed")

list(PROC.glob("*.csv"))


[WindowsPath('../data/processed/gdp_per_capita_tidy.csv'),
 WindowsPath('../data/processed/house_price_index_tidy.csv'),
 WindowsPath('../data/processed/inflation_hicp_tidy.csv'),
 WindowsPath('../data/processed/net_earnings_tidy.csv'),
 WindowsPath('../data/processed/unemployment_rate_tidy.csv')]

## 1) Load tidy CSVs

These files already have readable columns:
- `country`, `year`
- dataset-specific metric
- `unit` (kept for reference)


In [22]:
hpi   = pd.read_csv(PROC / "house_price_index_tidy.csv")
earn  = pd.read_csv(PROC / "net_earnings_tidy.csv")
unemp = pd.read_csv(PROC / "unemployment_rate_tidy.csv")
hicp  = pd.read_csv(PROC / "inflation_hicp_tidy.csv")
gdp   = pd.read_csv(PROC / "gdp_per_capita_tidy.csv")

hpi.head(10)

,country,year,value,unit
0,Austria,2015,128.63,"Annual average index, 2010=100"
1,Austria,2016,136.27,"Annual average index, 2010=100"
2,Austria,2017,144.48,"Annual average index, 2010=100"
3,Austria,2018,153.57,"Annual average index, 2010=100"
4,Austria,2019,162.82,"Annual average index, 2010=100"
5,Austria,2020,175.30,"Annual average index, 2010=100"
6,Austria,2021,198.25,"Annual average index, 2010=100"
7,Austria,2022,221.28,"Annual average index, 2010=100"
8,Austria,2023,212.27,"Annual average index, 2010=100"
9,Austria,2024,208.89,"Annual average index, 2010=100"


## 2) Standardize country names and drop aggregates

From Eurostat, the `country` column contains:
- **Actual countries** (Austria, Bulgaria, …)
- **Aggregates** (e.g., *European Union - 27 countries*, *Euro area – 20 countries*).

We keep only real countries.  
Then we standardize naming differences (e.g., "Turkey" → "Türkiye").


In [23]:
import re

AGGREGATE_PAT = re.compile(r"^(European Union|Euro area)", flags=re.IGNORECASE)

def drop_aggregates(df: pd.DataFrame, col: str = "country") -> pd.DataFrame:
    out = df.copy()
    out[col] = out[col].astype(str).str.strip()
    return out[~out[col].str.match(AGGREGATE_PAT, na=False)].copy()

COUNTRY_MAP = {
    "Czech Republic": "Czechia",
    "Turkey": "Türkiye",
    "UK": "United Kingdom",
}

def standardize_country_names(df: pd.DataFrame, col: str = "country") -> pd.DataFrame:
    out = df.copy()
    out[col] = out[col].astype(str).str.strip().replace(COUNTRY_MAP)
    return out

def clean_geo(df: pd.DataFrame) -> pd.DataFrame:
    return standardize_country_names(drop_aggregates(df))

hpi   = clean_geo(hpi)
earn  = clean_geo(earn)
unemp = clean_geo(unemp)
hicp  = clean_geo(hicp)
gdp   = clean_geo(gdp)

sorted(hpi["country"].unique())

['Austria',
 'Belgium',
 'Bulgaria',
 'Croatia',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Hungary',
 'Ireland',
 'Italy',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'Netherlands',
 'Norway',
 'Poland',
 'Portugal',
 'Romania',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden',
 'Türkiye',
 'United Kingdom']

## 3) Ensure types & select the columns we need

- `year` → integer (nullable Int64)
- metric columns → numeric


In [24]:
hpi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 285 entries, 0 to 339
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  285 non-null    object 
 1   year     285 non-null    int64  
 2   value    285 non-null    float64
 3   unit     285 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 11.1+ KB


## 4) Rename "value" and Merge into a single country–year panel

We **inner-join** on (`country`, `year`) to keep the intersection where all metrics exist.
We also compute **real_earnings** = net_earnings adjusted by HICP index:


In [28]:
def rename_metric(df, metric_name):
    out = df.rename(columns={"value": metric_name})
    return out[["country", "year", metric_name]].copy()

hpi_   = rename_metric(hpi,   "house_price_index")
earn_  = rename_metric(earn,  "net_earnings")
unemp_ = rename_metric(unemp, "unemployment_rate")
hicp_  = rename_metric(hicp,  "hicp_index")
gdp_   = rename_metric(gdp,   "gdp_per_capita")


In [29]:
panel = (
    hpi_
    .merge(earn_,  on=["country","year"], how="inner")
    .merge(unemp_, on=["country","year"], how="inner")
    .merge(hicp_,  on=["country","year"], how="inner")
    .merge(gdp_,   on=["country","year"], how="inner")
    .sort_values(["country","year"])
    .reset_index(drop=True)
)

panel["real_earnings"] = panel["net_earnings"] / (panel["hicp_index"] / 100.0)
panel.head(), panel.shape

(   country  year  house_price_index  net_earnings  unemployment_rate  \
 0  Austria  2015             128.63      34414.48                6.1   
 1  Austria  2016             136.27      36707.82                6.5   
 2  Austria  2017             144.48      37220.14                5.9   
 3  Austria  2018             153.57      37847.53                5.2   
 4  Austria  2019             162.82      41162.12                4.8   
 
    hicp_index  gdp_per_capita  real_earnings  
 0      100.00         39640.0   34414.480000  
 1      100.97         40690.0   36355.174804  
 2      103.22         41760.0   36059.038946  
 3      105.41         43360.0   35905.065933  
 4      106.98         44570.0   38476.462890  ,
 (277, 8))

## 5) Sanity checks

- Structure & basic stats  
- Duplicate keys  
- Missingness overview  
- Quick outlier scan (quantile-based)


In [30]:
panel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277 entries, 0 to 276
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   country            277 non-null    object 
 1   year               277 non-null    int64  
 2   house_price_index  277 non-null    float64
 3   net_earnings       277 non-null    float64
 4   unemployment_rate  277 non-null    float64
 5   hicp_index         277 non-null    float64
 6   gdp_per_capita     277 non-null    float64
 7   real_earnings      277 non-null    float64
dtypes: float64(6), int64(1), object(1)
memory usage: 17.4+ KB


In [32]:
panel.describe()

,year,house_price_index,net_earnings,unemployment_rate,hicp_index,gdp_per_capita,real_earnings
count,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000
mean,2019.512635,155.967726,26310.521227,6.696029,115.032274,34533.357401,23727.647021
std,2.859770,136.703917,15141.594373,2.960798,35.263878,23556.666060,13867.086976
min,2015.000000,71.090000,4860.670000,2.000000,98.680000,6560.000000,1535.765580
25%,2017.000000,108.420000,13130.480000,4.800000,101.690000,17270.000000,11583.462483
50%,2020.000000,136.300000,21858.940000,6.200000,106.270000,27850.000000,20684.987431
75%,2022.000000,175.000000,38662.770000,7.900000,119.040000,45380.000000,37057.010341
max,2024.000000,2066.460000,67125.600000,22.100000,570.940000,126910.000000,56288.822808


In [33]:
dups = panel.duplicated(subset=["country","year"]).sum()
dups

np.int64(0)

In [34]:
panel.isna().sum().sort_values(ascending=False)


country              0
year                 0
house_price_index    0
net_earnings         0
unemployment_rate    0
hicp_index           0
gdp_per_capita       0
real_earnings        0
dtype: int64

In [38]:
missing_by_country = (
    panel.assign(any_na=panel.isna().any(axis=1))
          .groupby("country")["any_na"]
          .sum()
          .sort_values(ascending=False)
)
missing_by_country.head(10)

country
Austria     0
Belgium     0
Bulgaria    0
Croatia     0
Cyprus      0
Czechia     0
Denmark     0
Estonia     0
Finland     0
France      0
Name: any_na, dtype: int64

## 6) Quick outlier scan (99th percentile by metric)

This is **diagnostic only** (we don’t drop; we just list the top values).

In [39]:
metrics = ["house_price_index","net_earnings","unemployment_rate","hicp_index","gdp_per_capita","real_earnings"]
q = panel[metrics].quantile(0.99).to_dict()
q

mask = False
for m in metrics:
    mask = mask | (panel[m] > q[m])

panel_outliers = panel.loc[mask, ["country","year"] + metrics].sort_values(["house_price_index"], ascending=False)
panel_outliers.head(12)


,country,year,house_price_index,net_earnings,unemployment_rate,hicp_index,gdp_per_capita,real_earnings
276,Türkiye,2023,2066.46,8768.30,9.4,570.94,12080.0,1535.765580
275,Türkiye,2022,1046.23,6285.77,10.5,370.82,10150.0,1695.100049
274,Türkiye,2021,412.75,5998.90,12.0,215.24,8200.0,2787.074893
165,Luxembourg,2022,236.72,62822.17,4.6,118.55,117100.0,52992.129903
164,Luxembourg,2021,217.49,60380.97,5.3,109.61,113920.0,55087.099717
166,Luxembourg,2023,210.47,65427.08,5.2,122.02,121290.0,53619.963940
167,Luxembourg,2024,197.73,67125.60,6.4,124.77,126910.0,53799.471027
163,Luxembourg,2020,189.21,59626.75,6.8,105.93,102190.0,56288.822808
160,Luxembourg,2017,138.08,55428.25,5.5,102.15,97440.0,54261.625061
250,Spain,2017,78.78,22857.77,17.2,101.69,25160.0,22477.893598


## 7) Save outputs

We save both CSV and Parquet for downstream notebooks.


In [40]:
out_csv = PROC / "merged_panel.csv"
out_parq = PROC / "merged_panel.parquet"

panel.to_csv(out_csv, index=False, encoding="utf-8")
panel.to_parquet(out_parq, index=False)

out_csv, out_parq

(WindowsPath('../data/processed/merged_panel.csv'),
 WindowsPath('../data/processed/merged_panel.parquet'))

# Data Preparation — Notes for the report

We successfully built a **clean country–year panel** from Eurostat data (2015–2024).

- **Countries included (29):** Austria, Belgium, Bulgaria, Croatia, Cyprus, Czechia, Denmark, Estonia, Finland, France, Germany, Hungary, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Netherlands, Norway, Poland, Portugal, Romania, Slovakia, Slovenia, Spain, Sweden, Türkiye, United Kingdom.

- **Shape:** 277 rows × 8 columns  
  (`country, year, house_price_index, net_earnings, unemployment_rate, hicp_index, gdp_per_capita, real_earnings`)

- **Missing data:** None (all columns are fully populated for these countries/years).

- **Outliers (diagnostic only):**
  - **Türkiye:** extremely high HPI (up to 2066) and very high inflation (HICP > 500).  
  - **Luxembourg:** very high net earnings (>65k EUR) and GDP per capita (>120k EUR).  
  - **Spain (2015–2017):** unusually low HPI values (~71–79) compared to peers.  

These outliers are kept in the dataset, but they will need **careful handling in plots and regressions** (e.g. log-scaling or trimming).

- **Saved outputs:**  
  - `data/processed/merged_panel.csv`  
  - `data/processed/merged_panel.parquet`  

---

**Next step:**  
Move to `02_exploration.ipynb` to analyze **Europe-wide trends** and then focus on **Bulgaria**.
